In [10]:
import glob
import os
import pandas as pd

csv_files = glob.glob("ativos/*.csv")

# Vamos montar uma lista de dicionários com Ticker e Origem
resultados = []

for fn in csv_files:
    nome_arquivo = os.path.basename(fn)
    df = pd.read_csv(fn, header=None, dtype=str)

    # separa em dois pares (Ticker, Cor) conforme antes
    df1 = df[[0, 2]].rename(columns={0: "Ticker", 2: "Cor"})
    df2 = df[[5, 7]].rename(columns={5: "Ticker", 7: "Cor"})
    df_all = pd.concat([df1, df2], ignore_index=True)

    df_all["Ticker"] = df_all["Ticker"].str.strip()
    df_all["Cor"]    = df_all["Cor"].str.strip().str.lower()

    # filtra só os verdes e registra a origem
    verdes = df_all.loc[df_all["Cor"] == "vermelho", "Ticker"]
    for ticker in verdes:
        resultados.append({
            "Ticker": ticker,
            "Setor": nome_arquivo
        })

# transforma em DataFrame para visualizar
df_res = pd.DataFrame(resultados)
print(df_res)


    Ticker                                Setor
0      EDU  Academic & Educational Services.csv
1      TAL  Academic & Educational Services.csv
2      GHC  Academic & Educational Services.csv
3     AFYA  Academic & Educational Services.csv
4     GOTU  Academic & Educational Services.csv
..     ...                                  ...
179    RCI                       Technology.csv
180    EBR                        Utilities.csv
181    BEP                        Utilities.csv
182    BEP                        Utilities.csv
183    EBR                        Utilities.csv

[184 rows x 2 columns]


In [4]:
import glob
import pandas as pd

csv_files = glob.glob("ativos/*.csv")

verdes = []
amarelos = []
vermelhos = []

for fn in csv_files:
    # 1) Lê pulando a primeira linha (header) e pega só as 2 colunas necessárias
    df = pd.read_csv(
        fn,
        header=None,          # diz que não vamos usar o cabeçalho do arquivo
        skiprows=1,           # pula a linha que contém TAMAN,SCORE,COR
        usecols=[0,1],        # coluna 0 = ticker, coluna 1 = score
        names=["Ticker","Score"],  # nomes das colunas no DataFrame
        decimal=","           # para converter "38,24" → 38.24
    )
    
    # 2) Converte Score para float (já cuida da vírgula)
    df["Score"] = df["Score"].astype(float)
    
    # 3) Classifica em vermelho/amarelo/verde
    df["Cor"] = pd.cut(
        df["Score"],
        bins=[-float("inf"), 40, 60, float("inf")],
        labels=["vermelho","amarelo","verde"]
    )
    
    # 4) Acumula em cada lista
    verdes   .extend(df.loc[df["Cor"]=="verde",    "Ticker"].tolist())
    amarelos .extend(df.loc[df["Cor"]=="amarelo",  "Ticker"].tolist())
    vermelhos.extend(df.loc[df["Cor"]=="vermelho", "Ticker"].tolist())

# 5) (Opcional) Remove duplicatas mantendo ordem
verdes    = list(dict.fromkeys(verdes))
amarelos  = list(dict.fromkeys(amarelos))
vermelhos = list(dict.fromkeys(vermelhos))

print("VERDES   :", verdes)
print("AMARELOS :", amarelos)
print("VERMELHOS:", vermelhos)


VERDES   : ['IP', 'MCD', 'LOW', 'TJX', 'CMG', 'RACE', 'ORLY', 'YUM', 'PM', 'MNST', 'AON', 'MCK', 'WM', 'RSG', 'CARR', 'GWW', 'PAYX', 'FAST', 'WAB', 'LUV', 'EQIX', 'O', 'BEKE', 'INVH', 'V', 'CRM', 'TMUS', 'IBM', 'ADI', 'APH']
AMARELOS : ['LRN', 'LOPE', 'ATGE', 'LAUR', 'STRA', 'PRDO', 'UTI', 'FC', 'APEI', 'VSTA', 'RIO', 'SHW', 'APD', 'SCCO', 'ECL', 'FCX', 'AEM', 'CTVA', 'VMC', 'MLM', 'PPG', 'NTR', 'LYB', 'PKG', 'MT', 'STLD', 'BALL', 'RS', 'AVY', 'WLK', 'GFI', 'JHX', 'AMCR', 'AU', 'AMZN', 'TSLA', 'COST', 'HD', 'DIS', 'BKNG', 'SBUX', 'NKE', 'MAR', 'RCL', 'TGT', 'HLT', 'AZO', 'LULU', 'ROST', 'TCOM', 'F', 'CUK', 'CCL', 'LYV', 'DECK', 'LVS', 'TSCO', 'WSM', 'NVR', 'BRK-A', 'BRK-B', 'PG', 'KO', 'PEP', 'HON', 'BUD', 'MMM', 'MDLZ', 'CL', 'KR', 'KMB', 'KDP', 'KVUE', 'CCEP', 'XOM', 'CVX', 'SHEL', 'COP', 'TTE', 'ENB', 'EOG', 'EPD', 'ET', 'WMB', 'KMI', 'SLB', 'OKE', 'MPLX', 'LNG', 'FANG', 'SU', 'OXY', 'HES', 'BAC', 'WFC', 'MS', 'AXP', 'GS', 'RY', 'BLK', 'SCHW', 'PGR', 'CB', 'MMC', 'APO', 'ICE', 'NWG'

In [12]:
import glob
import os
import pandas as pd

# 1) Liste todos os CSVs dentro de ativos/
csv_files = glob.glob("ativos/*.csv")

# 2) Acumula todos os registros aqui
dados = []

for fn in csv_files:
    # captura o nome do CSV atual
    nome_arquivo = os.path.basename(fn)
    
    df = pd.read_csv(
        fn,
        header=None,
        skiprows=1,            # pula a linha de cabeçalho original
        usecols=[0,1],         # coluna 0 = Ticker, 1 = Score
        names=["Ticker","Score"],
        decimal=","            # converte "38,24" → 38.24
    )
    # 3) Classificação por cor
    df["Cor"] = pd.cut(
        df["Score"],
        bins=[-float("inf"), 40, 60, float("inf")],
        labels=["vermelho","amarelo","verde"]
    )
    # 4) Adiciona linha a linha na lista, agora com o setor correto
    for _, row in df.iterrows():
        dados.append({
            "Ticker": row["Ticker"].strip(),
            "Score":  row["Score"],
            "Cor":    row["Cor"],
            "Setor":  nome_arquivo
        })

# 5) Converte para DataFrame único
df_todos = pd.DataFrame(dados)

# 6) Ordena decrescente por Score
df_todos = df_todos.sort_values("Score", ascending=False).reset_index(drop=True)

# 7) Exibe ou exporta
print(df_todos)
df_todos.to_csv("ativos_classificados_ordenados.csv", index=False)


    Ticker  Score       Cor                                Setor
0      CRM  73.88     verde                       Technology.csv
1     RACE  70.87     verde               Consumer Cyclicals.csv
2     CARR  69.16     verde                        INDUSTRIA.csv
3      YUM  67.83     verde               Consumer Cyclicals.csv
4     EQIX  67.49     verde                      Real Estate.csv
..     ...    ...       ...                                  ...
348   FEDU  19.00  vermelho  Academic & Educational Services.csv
349    ADD  17.00  vermelho  Academic & Educational Services.csv
350     JZ  17.00  vermelho  Academic & Educational Services.csv
351   GSUN  16.50  vermelho  Academic & Educational Services.csv
352    COE  14.50  vermelho  Academic & Educational Services.csv

[353 rows x 4 columns]
